In [2]:
import os
import pandas as pd
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler

import librosa
import librosa.display

from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

% matplotlib inline
np.random.seed(7)

UsageError: Line magic function `%` not found.


In [3]:
base_dir = '../data/'
train_dir = base_dir+'train/'
val_dir = base_dir+'val/'
test_dir = base_dir+'test/'

In [4]:
def load_wavs(filenames):
    return np.asarray([librosa.load(filename)[0] for filename in tqdm(filenames)])

### If you have lack of memory, Use this
#     wav = librosa.load(filenames[0])
#     wavs = np.zeros( (len(filenames), wav.shape[0]) )
#     for i, filename in enumerate(filenames):
#         wavs[i][:] = librosa.load(filename)[:]
#     return wavs
    
def find_y_by_filename(filename, y_dict):
    basename = os.path.basename(filename)
    y = y_dict[basename]
    return y

def make_y_by_filenames(filenames, y_dict):
    return np.asarray([find_y_by_filename(filename, y_dict) 
                           for filename in filenames])

def make_xy_data(filenames, y_dict):
    x_train = load_wavs(filenames)
    y_train = make_y_by_filenames(filenames, y_dict)
    return x_train, y_train

In [5]:
### Make Y data
annotations_filename = 'data_annotations.csv'
df = pd.read_csv(annotations_filename)
y_dict = {filename:int(label) for _, filename, label, _ in df.itertuples()}
# y_dict

In [6]:
print('Make train data.......')
x_train_wav_filenames = [train_dir+filename for filename in os.listdir(train_dir)
                            if filename.endswith('.wav')]
x_val_wav_filenames = [val_dir+filename for filename in os.listdir(val_dir)
                            if filename.endswith('.wav')]
x_train_wav_filenames += x_val_wav_filenames
x_train_wavs, y_train = make_xy_data(x_train_wav_filenames, y_dict)

print(x_train_wavs.shape, y_train.shape)

Make train data.......


100%|██████████| 2257/2257 [10:22<00:00,  3.45it/s]


(2257, 110250) (2257,)


In [ ]:
print('Make test data.......')
x_test_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
x_test_wavs, y_test = make_xy_data(x_test_wav_filenames, y_dict)

In [223]:
def preprocess(wav):
    wav = sklearn.preprocessing.maxabs_scale(wav)
    wav_mfcc = librosa.feature.mfcc(y=wav, n_mfcc=13)
    wav_mfcc_std = StandardScaler().fit_transform(wav_mfcc)
    wav_mfcc_std_mean = wav_mfcc_std.mean(axis=1)

    features = np.concatenate([wav_mfcc_std_mean])
    return features

def train(x_train_wavs, y_train):
    x_train = np.apply_along_axis(preprocess, 1, x_train_wavs)

    logreg = linear_model.LogisticRegression(penalty='l2', C=0.5)
    logreg.fit(x_train, y_train)
    print('Logistic Regression Score:', logreg.score(x_train, y_train))
    joblib.dump(logreg, 'logreg_chainsaw.pkl')
    
    linear_svc = LinearSVC()
    linear_svc.fit(x_train, y_train)
    print('Linear SVM Score:', linear_svc.score(x_train, y_train))
    joblib.dump(linear_svc, 'linear_svc_chainsaw.pkl') 

    kernel_svc = SVC()
    kernel_svc.fit(x_train, y_train)
    print('Kernel SVM Score:', kernel_svc.score(x_train, y_train))
    joblib.dump(kernel_svc, 'kernel_svc_chainsaw.pkl')

    print()

def test(x_test_wavs, y_test):
    x_test = np.apply_along_axis(preprocess, 1, x_test_wavs)

    clf = joblib.load('logreg_chainsaw.pkl')
    %timeit -n 10 -r 10 clf.predict(x_test)
    y_test_estimated = clf.predict(x_test)
    print('Logistic Regression Accuracy:', accuracy_score(y_test_estimated, y_test))
    print('Logistic Regression F1 Score:', f1_score(y_test, y_test_estimated))
    print()
    
    clf = joblib.load('linear_svc_chainsaw.pkl')
    %timeit -n 10 -r 10 clf.predict(x_test)
    y_test_estimated = clf.predict(x_test)
    print('Linear SVM Accuracy:', accuracy_score(y_test_estimated, y_test))
    print('Linear SVM F1 Score:', f1_score(y_test, y_test_estimated))
    print()
    
    clf = joblib.load('kernel_svc_chainsaw.pkl')
    x_test = np.apply_along_axis(preprocess, 1, x_test_wavs)
    %timeit -n 10 -r 10 clf.predict(x_test)
    y_test_estimated = clf.predict(x_test)
    print('Kernel SVM Accuracy:', accuracy_score(y_test_estimated, y_test))
    print('Kernel SVM F1 Score:', f1_score(y_test, y_test_estimated))
    print()

In [224]:
train(x_train_wavs, y_train)
test(x_test_wavs, y_test)

c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Score: 0.9308817013735047
Linear SVM Score: 0.9308817013735047


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Kernel SVM Score: 0.9481612760301285

169 µs ± 13.7 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Logistic Regression Accuracy: 0.9354838709677419
Logistic Regression F1 Score: 0.9470404984423676

340 µs ± 138 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Linear SVM Accuracy: 0.9307400379506642
Linear SVM F1 Score: 0.9432789432789432

15.6 ms ± 1.58 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
Kernel SVM Accuracy: 0.9516129032258065
Kernel SVM F1 Score: 0.9606784888203548



In [225]:
base_dir = '../ESC-50-master/split_wav/'
test_dir = base_dir+'wind/'

print('Make test data.......')
x_temp_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
# x_temp_wavs, y_temp = make_xy_data(x_test_wav_filenames, y_dict)

x_temp_wavs = load_wavs(x_temp_wav_filenames)
y_temp = np.zeros(x_temp_wavs.shape[0])

test(x_temp_wavs, y_temp)


base_dir = '../ESC-50-master/split_wav/'
test_dir = base_dir+'rain/'

print('Make test data.......')
x_temp_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
# x_temp_wavs, y_temp = make_xy_data(x_test_wav_filenames, y_dict)

x_temp_wavs = load_wavs(x_temp_wav_filenames)
y_temp = np.zeros(x_temp_wavs.shape[0])

test(x_temp_wavs, y_temp)

base_dir = '../ESC-50-master/split_wav/'
test_dir = base_dir+'engine/'

print('Make test data.......')
x_temp_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
# x_temp_wavs, y_temp = make_xy_data(x_test_wav_filenames, y_dict)

x_temp_wavs = load_wavs(x_temp_wav_filenames)
y_temp = np.zeros(x_temp_wavs.shape[0])

test(x_temp_wavs, y_temp)

Make test data.......


66.6 µs ± 20.5 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Logistic Regression Accuracy: 0.85


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Logistic Regression F1 Score: 0.0

The slowest run took 4.31 times longer than the fastest. This could mean that an intermediate result is being cached.
130 µs ± 63.3 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Linear SVM Accuracy: 0.85


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


...

Logistic Regression F1 Score: 0.0

183 µs ± 57.3 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Linear SVM Accuracy: 0.65


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Linear SVM F1 Score: 0.0

838 µs ± 121 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Kernel SVM Accuracy: 0.65


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Kernel SVM F1 Score: 0.0



In [227]:
def preprocess(wav):
    wav = sklearn.preprocessing.maxabs_scale(wav)
    wav_mfcc = librosa.feature.mfcc(y=wav, n_mfcc=13)
#     wav_mfcc_std = StandardScaler().fit_transform(wav_mfcc)
    wav_mfcc_std_mean = wav_mfcc.mean(axis=1)

    S = librosa.feature.melspectrogram(wav, n_mels=128)
    log_S = librosa.amplitude_to_db(S)    
#     log_S_std = StandardScaler().fit_transform(log_S)
    log_S_std_mean = log_S.mean(axis=1)
    
    features = np.concatenate([wav_mfcc_std_mean, log_S_std_mean])
    return features

def train(x_train_wavs, y_train):
    x_train = np.apply_along_axis(preprocess, 1, x_train_wavs)

    logreg = linear_model.LogisticRegression(C=0.01)
    logreg.fit(x_train, y_train)
    print('Logistic Regression Score:', logreg.score(x_train, y_train))
    joblib.dump(logreg, 'logreg_chainsaw.pkl')
    
    linear_svc = LinearSVC()
    linear_svc.fit(x_train, y_train)
    print('Linear SVM Score:', linear_svc.score(x_train, y_train))
    joblib.dump(linear_svc, 'linear_svc_chainsaw.pkl') 

    kernel_svc = SVC()
    kernel_svc.fit(x_train, y_train)
    print('Kernel SVM Score:', kernel_svc.score(x_train, y_train))
    joblib.dump(kernel_svc, 'kernel_svc_chainsaw.pkl')

    print()

def test(x_test_wavs, y_test):
    x_test = np.apply_along_axis(preprocess, 1, x_test_wavs)

    clf = joblib.load('logreg_chainsaw.pkl')
    %timeit -n 10 -r 10 clf.predict(x_test)
    y_test_estimated = clf.predict(x_test)
    print('Logistic Regression Accuracy:', accuracy_score(y_test_estimated, y_test))
    print('Logistic Regression F1 Score:', f1_score(y_test, y_test_estimated))
    print()
    
    clf = joblib.load('linear_svc_chainsaw.pkl')
    %timeit -n 10 -r 10 clf.predict(x_test)
    y_test_estimated = clf.predict(x_test)
    print('Linear SVM Accuracy:', accuracy_score(y_test_estimated, y_test))
    print('Linear SVM F1 Score:', f1_score(y_test, y_test_estimated))
    print()
    
    clf = joblib.load('kernel_svc_chainsaw.pkl')
    x_test = np.apply_along_axis(preprocess, 1, x_test_wavs)
    %timeit -n 10 -r 10 clf.predict(x_test)
    y_test_estimated = clf.predict(x_test)
    print('Kernel SVM Accuracy:', accuracy_score(y_test_estimated, y_test))
    print('Kernel SVM F1 Score:', f1_score(y_test, y_test_estimated))
    print()

In [228]:
train(x_train_wavs, y_train)
test(x_test_wavs, y_test)

c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Score: 0.9654408506867523


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Linear SVM Score: 0.9534780682321666


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Kernel SVM Score: 1.0

353 µs ± 65.7 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Logistic Regression Accuracy: 0.969639468690702
Logistic Regression F1 Score: 0.9750390015600624

The slowest run took 5.70 times longer than the fastest. This could mean that an intermediate result is being cached.
787 µs ± 376 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Linear SVM Accuracy: 0.9421252371916509
Linear SVM F1 Score: 0.9528957528957529

725 ms ± 34.3 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
Kernel SVM Accuracy: 0.6015180265654649
Kernel SVM F1 Score: 0.7511848341232228



In [229]:
base_dir = '../ESC-50-master/split_wav/'
test_dir = base_dir+'wind/'

print('Make test data.......')
x_temp_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
# x_temp_wavs, y_temp = make_xy_data(x_test_wav_filenames, y_dict)

x_temp_wavs = load_wavs(x_temp_wav_filenames)
y_temp = np.zeros(x_temp_wavs.shape[0])

test(x_temp_wavs, y_temp)


base_dir = '../ESC-50-master/split_wav/'
test_dir = base_dir+'rain/'

print('Make test data.......')
x_temp_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
# x_temp_wavs, y_temp = make_xy_data(x_test_wav_filenames, y_dict)

x_temp_wavs = load_wavs(x_temp_wav_filenames)
y_temp = np.zeros(x_temp_wavs.shape[0])

test(x_temp_wavs, y_temp)

base_dir = '../ESC-50-master/split_wav/'
test_dir = base_dir+'engine/'

print('Make test data.......')
x_temp_wav_filenames = [test_dir+filename for filename in os.listdir(test_dir)
                            if filename.endswith('.wav')]
# x_temp_wavs, y_temp = make_xy_data(x_test_wav_filenames, y_dict)

x_temp_wavs = load_wavs(x_temp_wav_filenames)
y_temp = np.zeros(x_temp_wavs.shape[0])

test(x_temp_wavs, y_temp)

Make test data.......


65.3 µs ± 14.7 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Logistic Regression Accuracy: 0.9


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Logistic Regression F1 Score: 0.0

75.1 µs ± 17.3 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Linear SVM Accuracy: 0.875


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


...

Logistic Regression F1 Score: 0.0

76.4 µs ± 12.5 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
Linear SVM Accuracy: 0.975


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Linear SVM F1 Score: 0.0

27.4 ms ± 4.04 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
Kernel SVM Accuracy: 0.65


c:\users\bbchip\desktop\project\chainsaw_classfication\vir_chainsaw\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Kernel SVM F1 Score: 0.0



In [231]:
preprocess(x_temp_wavs[0]).shape

(141,)